In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement

# Define the path to the ImageNet dataset
dataset_path = "D:\data\imagenet"

# Get the list of class folders in the dataset
class_folders = glob.glob(dataset_path+'/*/')

In [33]:
#img_folders[0]

In [12]:
import csv
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern

def extract_features_from_directory(directory, csv_file):
    # LBP parameters
    radius = 3
    n_points = 8 * radius  # Adjusted to ensure 24 points for LBP
    METHOD = 'uniform'
    num_bins = 256  # Explicitly set bins to 100 for the histogram

    image_paths = glob.glob(directory + '/*')
    histograms = []  # List to hold histogram data for each image

    for img_path in tqdm(image_paths):
        image = Image.open(img_path)
        
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        image = np.array(image.resize((32, 32))).astype("float32") / 255.0
        image_gray = rgb2gray(image)
        lbp = local_binary_pattern(image_gray, n_points, radius, METHOD)
        hist, _ = np.histogram(lbp, density=True, bins=num_bins, range=(0, n_points + 2))
        histograms.append(hist)
        #print(np.shape(histograms))

    # Average histogram across all images
    avg_feature = np.mean(histograms, axis=0)
    print(np.shape(avg_feature))

    
    with open(csv_file, 'a') as file:
        
        file.write(directory + '\t')
        
        for value in avg_feature:
            file.write(str(value) + '\t')
        file.write('\n')


In [13]:
from tqdm import tqdm
import csv

file_path = 'D:/lbpfeature.csv'
#header = ["Directory"] + ["Name"] + ['Feature {}'.format(i) for i in range(2048)]  # Example header
header = ["Directory"] + ['Feature {}'.format(i) for i in range(256)]  # Example header

with open(file_path, 'w') as file:
    
    for value in header:
        file.write(str(value) + '\t')
    file.write('\n')

z = None

for k in tqdm(range(len(class_folders[:3]))):
    extract_features_from_directory(class_folders[k], file_path)


 33%|██████████████████████▎                                            | 1/3 [00:05<00:11,  5.97s/it]

(256,)



 67%|████████████████████████████████████████████▋                      | 2/3 [00:11<00:05,  5.51s/it]

(256,)



100%|███████████████████████████████████████████████████████████████████| 3/3 [00:15<00:00,  5.33s/it]

(256,)


In [14]:
import pandas as pd

# Define the file path
file_path = r'D:/lbpfeature.csv'

# Read the tab-separated CSV file into a DataFrame
data = pd.read_csv(file_path, delimiter='\t')

print(len(data))

# Display the head of the DataFrame
print(data.shape)


3
(3, 258)
